In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr


In [4]:
path = "/home/mpiuser/Datasets/air.*.nc"

ds = xr.open_mfdataset(path)
ds

<xarray.Dataset>
Dimensions:  (lat: 73, level: 17, lon: 144, time: 3287)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * time     (time) datetime64[ns] 2002-01-01 2002-01-02 ... 2010-12-31
Data variables:
    air      (time, level, lat, lon) float32 dask.array<chunksize=(365, 17, 73, 144), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (2002)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 03/08/18 by Hoop (netCDF2.3)\nConverted to chunke...
    References:     http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reana...
    dataset_title:  NCEP-NCAR Reanalysis 1

In [5]:
air=ds["air"]
air

<xarray.DataArray 'air' (time: 3287, level: 17, lat: 73, lon: 144)>
dask.array<concatenate, shape=(3287, 17, 73, 144), dtype=float32, chunksize=(366, 17, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * time     (time) datetime64[ns] 2002-01-01 2002-01-02 ... 2010-12-31
Attributes:
    long_name:    mean Daily Air temperature
    units:        degK
    precision:    2
    GRIB_id:      11
    GRIB_name:    TMP
    var_desc:     Air temperature
    dataset:      NCEP Reanalysis Daily Averages
    level_desc:   Multiple levels
    statistic:    Mean
    parent_stat:  Individual Obs
    valid_range:  [150. 350.]

In [16]:
len_of_dim = air.sizes
time_size = len_of_dim[air.dims[0]]
rank = len(len_of_dim)


In [18]:
 if (rank != 4):
        print("Expected variable of rank = 3, recieved rank = {}".format(rank))
      
    

In [25]:
result=air.groupby('time.dayofyear').mean('time')
result

<xarray.DataArray 'air' (dayofyear: 366, level: 17, lat: 73, lon: 144)>
dask.array<stack, shape=(366, 17, 73, 144), dtype=float32, chunksize=(1, 17, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * level      (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366

In [27]:
result[66,:,12,12].values

array([268.08224, 266.94888, 264.4989 , 256.70667, 249.16002, 240.45334,
       230.2378 , 220.48557, 217.20332, 217.1778 , 217.67667, 215.17336,
       212.66446, 210.97113, 210.3489 , 212.65779, 218.52113],
      dtype=float32)

In [35]:
def clmDayTLLL(x):
    
    # Calculate the sizes of time dimension
    len_of_dim = x.sizes
    time_size = len_of_dim[x.dims[0]]
    
    
    
    # Get the sizes of latitude and longitude dimension
    lev_size = len_of_dim[x.dims[1]]
    lat_size = len_of_dim[x.dims[2]]
    lon_size = len_of_dim[x.dims[3]]
    
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[0]
    
    
    
    # Compute 12 months average
    ave_month = x.groupby("time.dayofyear").mean(time)
    
    # Copy and update the attributes
    ave_month.attrs = x.attrs
    ave_month.encoding=x.encoding
    ave_month.attrs['info'] = "function clmDayTLL"
    return ave_month
    
    # Return the results

In [36]:
result=clmDayTLLL(air)
result

<xarray.DataArray 'air' (dayofyear: 366, level: 17, lat: 73, lon: 144)>
dask.array<stack, shape=(366, 17, 73, 144), dtype=float32, chunksize=(1, 17, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * level      (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Attributes:
    long_name:    mean Daily Air temperature
    units:        degK
    precision:    2
    GRIB_id:      11
    GRIB_name:    TMP
    var_desc:     Air temperature
    dataset:      NCEP Reanalysis Daily Averages
    level_desc:   Multiple levels
    statistic:    Mean
    parent_stat:  Individual Obs
    valid_range:  [150. 350.]
    info:         function clmDayTLL